<a href="https://colab.research.google.com/github/DevAssis/LLM_RAG_Dados_LangChaim/blob/master/LLM_RAG_dados_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color=blue> LLM's com RAG

# Pipeline para Converter Dados Não Estruturados em Perguntas e Respostas

Este notebook implementa um pipeline de Processamento de Linguagem Natural (PNL) para transformar dados não estruturados em um sistema de perguntas e respostas interativo. O pipeline utiliza técnicas de embedding, armazenamento vetorial e modelos de linguagem avançados para permitir que os usuários obtenham insights a partir de seus dados.

## Etapas do Pipeline

1. **Carregamento de Dados**: Os dados não estruturados são carregados em um formato adequado para processamento.

2. **Divisão do Texto**: O texto é dividido em unidades menores para facilitar a análise e o gerenciamento.

3. **Embedding e Armazenamento**: As unidades de texto são convertidas em embeddings vetoriais e armazenadas em um banco de dados vetorial para busca eficiente.

4. **Recuperação e Geração**: O sistema recupera as informações relevantes do banco de dados e gera respostas concisas e informativas para as perguntas dos usuários.

# Etapa 1: Carregamento do Documento

Nesta etapa, as lib's são instaladas e os dados não estruturados são carregados utilizando o LangChain. A fonte de dados pode variar, incluindo arquivos de texto, páginas da web, etc. O carregador selecionado converte os dados brutos em um objeto `Document` do LangChain, que será utilizado nas etapas subsequentes.

**Exemplo:**


```
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://bkjfiel.com.br/proverbios-1")
data = loader.load()
```



In [ ]:
# Instalar o pacote atualizado
!pip install -U langchain-openai

In [ ]:
!pip install langchain

In [ ]:
# Instalar as bibliotecas necessárias
!pip install -U langchain langchain-community openai faiss-cpu

In [ ]:
!pip install tiktoken

In [ ]:
!pip install sentence-transformers

##### Fetching API Key from Environment Variable

In [ ]:
!pip install python-dotenv

In [1]:
import os
# Fetch the OpenAI API key from environment variables
api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
import os

# Verificar se a chave da API foi recuperada corretamente
api_key = os.getenv('OPENAI_API_KEY')

# Imprimir o resultado (por segurança, evite imprimir toda a chave publicamente)
print(api_key is not None)  # Deve retornar True se a chave foi carregada corretamente


In [ ]:
from langchain.document_loaders import WebBaseLoader

# Initialize the WebBaseLoader with the URL of the document to be loaded
loader = WebBaseLoader("https://bkjfiel.com.br/proverbios-1")

# Load the document and store it in the 'data' variable
data = loader.load()

In [ ]:
# Display the content of the loaded document
print(data)

# Etapa 2: Dividindo o Documento em Partes

O documento carregado é dividido em partes menores, chamadas de "chunks". Essa divisão facilita o processamento e a geração de embeddings. O tamanho dos chunks e a sobreposição entre eles são definidos para garantir que o contexto seja mantido.

**Exemplo:**



```
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len
)

chunks = text_splitter.split_documents(data)
```



In [ ]:
import tiktoken

# Set up token encoding for the GPT-3.5 Turbo model
tiktoken.encoding_for_model('gpt-4o-mini')

In [ ]:
tokenizer = tiktoken.get_encoding('o200k_base')

# Define a function to calculate the token length of a given text
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("O temor do Senhor é o princípio da Sabedoria.")

In [ ]:
# Este código inicializa um divisor de texto utilizando o `RecursiveCharacterTextSplitter`,
# que divide o texto em partes (chunks) de tamanho especificado.
# Neste caso, o tamanho do chunk é de 100 caracteres, com uma sobreposição de 20 caracteres entre os chunks.
# A função `length_function` é usada para calcular o comprimento do texto usando `tiktoken_len`.

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Inicializar o divisor de texto com parâmetros especificados
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len
)


In [ ]:
# Split the loaded document into smaller chunks
chunks = text_splitter.split_documents(data)

In [ ]:
# Check the total number of chunks generated
len(chunks)

# Etapa 3: Armazenando as Embeddings Vetoriais no Banco de Dados Vetorial

Cada chunk de texto é convertido em uma representação vetorial (embedding) usando um modelo de embedding pré-treinado. Essas embeddings capturam o significado semântico do texto e permitem a busca por similaridade. As embeddings e os chunks são armazenados em um banco de dados vetorial (ChromaDB) para recuperação eficiente.

**Exemplo:**



```
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

hf = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs = {'device' : 'cpu'}
)

vectordb = Chroma.from_documents(chunks, hf)
```


In [ ]:
# O código abaixo inicializa embeddings utilizando um modelo do HuggingFace.
# O modelo utilizado é o "sentence-transformers/all-MiniLM-L6-v2", que é um modelo leve e eficiente
# para gerar embeddings de sentenças.
# O parâmetro `model_kwargs` especifica que o modelo será executado na CPU,
# enquanto `encode_kwargs` define que as embeddings não serão normalizadas.
# Após a configuração dos parâmetros, o HuggingFace Embeddings é inicializado.

from langchain.embeddings import HuggingFaceEmbeddings

# Especificar o nome do modelo e argumentos adicionais
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device' : 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Inicializar as Embeddings do HuggingFace
hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)


In [ ]:
embed = hf.embed_documents(texts=['h','e'])

# Print the length of one of the embeddings to check its dimensions
print(len(embed[1]))

In [ ]:
# Install ChromaDB package using pip
!pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

# Initialize Chroma vector database with chunks and HuggingFace embeddings
vectordb = Chroma.from_documents(chunks, hf)

In [ ]:
# Perform a similarity search on the vector database
vectordb.similarity_search('não mentir', k=3)

# Etapa 4: Recuperar e Gerar

Quando o usuário faz uma pergunta, o sistema busca no banco de dados vetorial os chunks mais relevantes (semelhantes à pergunta). Um modelo de linguagem (LLM) utiliza esses chunks e a pergunta do usuário para gerar uma resposta completa e informativa.

**Exemplo:**


```
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name='gpt-4', temperature=0.6)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

response = qa_chain({'query': 'Como devo viver minha vida na terra?'})
```




In [ ]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Recuperar a chave da OpenAI da aba Secrets
api_key = userdata.get('OPENAI_API_KEY')

# Verificar se a chave foi carregada corretamente
if not api_key:
    raise ValueError("Chave da API OpenAI não encontrada. Verifique se foi configurada corretamente na aba Secrets.")

# Inicializar o modelo de linguagem com a chave diretamente
llm = ChatOpenAI(model_name='gpt-4', temperature=0.6, openai_api_key=api_key)

# Inicializar a cadeia RetrievalQA com o modelo de linguagem e o retriever do banco de dados vetorial
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

# Passar uma consulta para a cadeia de QA para gerar uma resposta
response = qa_chain({'query': 'Como devo viver minha vida na terra?'})

# Exibir a resposta
print(response)


In [ ]:
import pprint

# Exibir a resposta de forma mais organizada
pprint.pprint(response)


In [ ]:
#Change the query to what you want to ask the LLM
query = 'Como devo agir com as pessoas?'

In [ ]:
qa_chain({'query' : query})

# Próximos Passos

* **Engenharia de Prompts**: Aperfeiçoar os prompts (instruções) fornecidos ao LLM para melhorar a qualidade e a relevância das respostas.

* **Prompt Templates**: Utilizar Prompt Templates do LangChain para criar prompts dinâmicos e reutilizáveis.

#FIM